In [7]:
import os
import logging
from clickhouse_driver import Client
from dotenv import load_dotenv
from datetime import datetime, timedelta, date

# Загрузка переменных окружения
load_dotenv()

# Настройка логирования
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    handlers=[
        logging.FileHandler("adjust_status.log", encoding='utf-8'),
        logging.StreamHandler()
    ]
)
logger = logging.getLogger()

# Глобальные переменные
database_name = os.getenv('CLICKHOUSE_DB', 'default')

# Подключение к ClickHouse
client = Client(
    host=os.getenv('CLICKHOUSE_HOST', '10.95.19.132'),
    user=os.getenv('CLICKHOUSE_USER', 'default'),
    password=os.getenv('CLICKHOUSE_PASSWORD', 'quie1ahpoo5Su0wohpaedae8keeph6bi'),
    port=9000,
    secure=False,
    settings={'strings_encoding': 'utf-8'}
)


def adjust_status():
    """
    Корректирует поле Status в таблице OlapCube_VNV на основе значения balance_total и статусов в Status_P.
    Использует target_date = initial_date + 1 день.
    """
    try:
        # Извлечь initial_date (самая ранняя дата)
        query_initial_date = f"""
        SELECT MIN(Dates) AS initial_date
        FROM {database_name}.OlapCube_VNV
        """
        initial_date_result = client.execute(query_initial_date)
        initial_date = initial_date_result[0][0]
        logger.info(f"Initial date in OlapCube_VNV: {initial_date}")

        # Проверка наличия initial_date
        if not initial_date:
            logger.error("Не удалось получить initial_date. Завершаем выполнение.")
            return

        # Вычислить target_date = initial_date + 1 день
        if isinstance(initial_date, datetime):
            initial_date_date = initial_date.date()
        elif isinstance(initial_date, date):
            initial_date_date = initial_date
        elif isinstance(initial_date, str):
            try:
                initial_date_date = datetime.strptime(initial_date, "%Y-%m-%d").date()
            except ValueError:
                logger.error(f"Неправильный формат initial_date: {initial_date}")
                return
        else:
            logger.error(f"Неизвестный формат initial_date: {initial_date}")
            return

        target_date = initial_date_date + timedelta(days=1)
        logger.info(f"Target date: {target_date}")

        # Получить значение balance_total на target_date
        query_balance_total = f"""
        SELECT balance_total
        FROM {database_name}.OlapCube_VNV
        WHERE Dates = '{target_date}'
        LIMIT 1
        """
        balance_total_result = client.execute(query_balance_total)
        if balance_total_result:
            balance_total = balance_total_result[0][0]
            logger.info(f"balance_total на target_date: {balance_total}")
        else:
            logger.error(f"balance_total не найден для даты {target_date}. Завершаем выполнение.")
            return

        # Извлечь записи Status_P на target_date
        query_status_p = f"""
        SELECT serialno, Status_P
        FROM {database_name}.OlapCube_VNV
        WHERE Dates = '{target_date}'
        ORDER BY serialno
        """
        status_p_data = client.execute(query_status_p)
        logger.info(f"Извлечено {len(status_p_data)} записей на target_date.")

        # Разделение записей по статусам
        exploitation = [row for row in status_p_data if row[1] == "Эксплуатация"]
        repaired = [row for row in status_p_data if row[1] == "Исправен"]
        inactive = [row for row in status_p_data if row[1] == "Неактивно"]

        logger.info(f"Начальное количество 'Эксплуатация': {len(exploitation)}")
        logger.info(f"Начальное количество 'Исправен': {len(repaired)}")
        logger.info(f"Начальное количество 'Неактивно': {len(inactive)}")

        changed_serialnos = []
        if balance_total == 0:
            # Копируем все значения Status_P в Status
            update_query = f"""
            ALTER TABLE {database_name}.OlapCube_VNV 
            UPDATE Status = Status_P
            WHERE Dates = '{target_date}'
            """
            client.execute(update_query)
            logger.info("Статусы скопированы без изменений.")
        elif balance_total > 0:
            # Меняем первых balance_total записей "Эксплуатация" на "Исправен"
            rows_to_update = exploitation[:int(balance_total)]
            for row in rows_to_update:
                update_query = f"""
                ALTER TABLE {database_name}.OlapCube_VNV 
                UPDATE Status = 'Исправен'
                WHERE serialno = '{row[0]}' AND Dates = '{target_date}'
                """
                client.execute(update_query)
                changed_serialnos.append(row[0])
            logger.info(f"Изменено {len(changed_serialnos)} записей: 'Эксплуатация' -> 'Исправен'. Serialno: {', '.join(changed_serialnos)}")
        elif balance_total < 0:
            # Меняем abs(balance_total) записей "Исправен" или "Неактивно" на "Эксплуатация"
            deficit_count = abs(int(balance_total))
            rows_to_add = repaired[:deficit_count]
            deficit_count -= len(rows_to_add)
            if deficit_count > 0:
                rows_to_add.extend(inactive[:deficit_count])
            for row in rows_to_add:
                update_query = f"""
                ALTER TABLE {database_name}.OlapCube_VNV 
                UPDATE Status = 'Эксплуатация'
                WHERE serialno = '{row[0]}' AND Dates = '{target_date}'
                """
                client.execute(update_query)
                changed_serialnos.append(row[0])
            logger.info(f"Изменено {len(changed_serialnos)} записей: 'Исправен' или 'Неактивно' -> 'Эксплуатация'. Serialno: {', '.join(changed_serialnos)}")

        # Копирование оставшихся значений из Status_P в Status
        update_remaining_query = f"""
        ALTER TABLE {database_name}.OlapCube_VNV
        UPDATE Status = Status_P
        WHERE Dates = '{target_date}' AND Status IS NULL
        """
        client.execute(update_remaining_query)
        logger.info("Оставшиеся значения скопированы из Status_P в Status.")

    except Exception as e:
        logger.error(f"Ошибка при корректировке статусов: {e}", exc_info=True)


def main():
    adjust_status()


if __name__ == "__main__":
    main()


2024-12-04 06:06:45,946 - INFO - Initial date in OlapCube_VNV: 2024-11-25
2024-12-04 06:06:45,947 - INFO - Target date: 2024-11-26
2024-12-04 06:06:45,956 - INFO - balance_total на target_date: 0
2024-12-04 06:06:45,993 - INFO - Извлечено 420 записей на target_date.
2024-12-04 06:06:45,995 - INFO - Начальное количество 'Эксплуатация': 156
2024-12-04 06:06:45,996 - INFO - Начальное количество 'Исправен': 20
2024-12-04 06:06:45,998 - INFO - Начальное количество 'Неактивно': 235
2024-12-04 06:06:46,005 - INFO - Статусы скопированы без изменений.
2024-12-04 06:06:46,022 - INFO - Оставшиеся значения скопированы из Status_P в Status.
